<a href="https://colab.research.google.com/github/pollinations/hive/blob/main/interesting_notebooks/COG_Swinir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://i.imgur.com/BKvDhnf.png" width="300" height="300" />

Uses [SwinIR](https://github.com/JingyunLiang/SwinIR) to perform 4x neural super-resolution (creates HD images from low resolution)

---
Credits: [Jingyun Liang]((https://github.com/JingyunLiang)
License: MIT

In [ ]:

# The image or video to scale up
image_file = ''  #@param {type: "string", accepted_files: "video,image"}

output_path = "/content"


In [ ]:
video_file = image_file

import mimetypes
mimetypes.init()

mimestart = mimetypes.guess_type(video_file)[0]
if mimestart is None:
    raise Exception("Could not determine file type of " + video_file)
mimestart = mimestart.split('/')[0]
is_video = mimestart == 'video'

input_path = "/tmp/sr_input"
!rm -rf $input_path
!mkdir -p $input_path

# Copy image or extract video frames and copy to input_path

if is_video:
    # Save video framerate
    framerate = !ffprobe -v error -select_streams v:0 -show_entries stream=r_frame_rate -of default=noprint_wrappers=1:nokey=1 "$video_file"
    multiplier, divisor = framerate[0].split('/')
    framerate = float(multiplier) / float(divisor)
    print("framerate", framerate)

    # Extract frames
    !ffmpeg -i "$video_file" $input_path/%05d.jpg
else:
    !cp "$video_file" $input_path/



In [ ]:
gpu, = !nvidia-smi -L
print(gpu)
if 'A100' in gpu:
  torch.backends.cudnn.enabled = False
  print('Finished setup for A100')

In [ ]:
%cd /content
!git clone https://github.com/voodoohop/SwinIR
%cd /content/SwinIR

In [ ]:
!wget -c "https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/003_realSR_BSRGAN_DFO_s64w8_SwinIR-M_x4_GAN.pth" -P experiments/pretrained_models


In [ ]:
#!pip install "torchvision==0.9.0" "torch==1.8.0" "numpy==1.19.4" "opencv-python==4.4.0.46" "tqdm==4.62.2" "Pillow==8.3.2" "timm==0.4.12"
!pip install timm


In [ ]:
#!pip install cog redis
#%cd /content/swinir

from predict import Predictor
p = Predictor()
p.setup()


In [ ]:
from glob import glob
from tqdm import tqdm

images = glob(f"{input_path}/*.jpg") + glob(f"{input_path}/*.png") + glob(f"{input_path}/*.jpeg")
images = list(sorted(images))

tmp_output = "/tmp/sr_output"
!rm -rf $tmp_output
!mkdir -p $tmp_output

for image_file in tqdm(images):

    #print("image_file", image_file)
    

    # Get the filename and extension
    filename = image_file.split('/')[-1]
    ext = filename.split('.')[-1]
   
    # Save to tmp_output
    out_file = f"{tmp_output}/{filename}"

    path = p.predict(image_file)
    !cp "{path}" "{out_file}"

    !cp "$out_file" $output_path/frame.{ext}

if is_video:
    # Create video
    !ffmpeg -framerate $framerate -i {tmp_output}/%05d.jpg -c:v libx264 -pix_fmt yuv420p $input_path/output.mp4
    
    # Check if original video contains audio stream
    has_audio_stream = !ffprobe -v error -select_streams a -show_entries stream=codec_name -of default=noprint_wrappers=1:nokey=1 "$video_file"
    print("has_audio_stream",has_audio_stream)
    has_audio_stream = len(has_audio_stream) > 0

    if has_audio_stream:
        # Combine with audio of original video
        !ffmpeg -i $input_path/output.mp4 -i "$video_file" -c copy -map 0:v -map 1:a -shortest $output_path/output.mp4
    
    else:
        # Copy output video
        !cp $input_path/output.mp4 $output_path/output.mp4
        
    import os.path
    if not os.path.exists(output_path + '/output.mp4'):
        raise Exception("Expected output file does not exist.")
    !rm {output_path}/*.jpg